In [1]:
from __future__ import division, absolute_import, print_function

import sys
import os
import pickle
import numpy as np
import random

import h5py
from sklearn import metrics 
from collections import Counter
from glob import glob
import gc
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from tables import *

from keras import backend as K 
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.backend import manual_variable_initialization 
from keras.models import load_model, Model
from keras.layers import Dense, concatenate, Flatten, Conv1D, BatchNormalization, Input, Dropout
from keras.optimizers import Adam
from keras.callbacks import CSVLogger, ModelCheckpoint, TerminateOnNaN
#import keras

#root
absPath = '/home/angela3/imbalance_pcm_benchmark/'
sys.path.insert(0, absPath)


from src.model_functions import *
from src.Target import Target
from src.postproc_auxiliar_functions import *

os.environ['PYTHONHASHSEED'] = '0' 
np.random.seed(8)
random.seed(8)
tf.random.set_seed(8)

Using TensorFlow backend.


In [2]:
absPath

'/home/angela3/imbalance_pcm_benchmark/'

In [3]:
nfolds = 10
batch_size = 128
epochss = 100
type_padding_prot = "pre_padding"
protein_type = "GPCRs" #"kinases"

In [4]:
#Loading maximum lengths of proteins and compounds
with open("".join((absPath, 'data/prot_max_len.pickle')), "rb") as input_file:
    max_len_prot = pickle.load(input_file)
#Defining protein dictionary    
instarget = Target("AAA")
prot_dict = instarget.predefining_dict()

In [5]:
learning_rate = 5e-4
decay_rate = learning_rate/epochss
adamm = Adam(lr=learning_rate, beta_1=0.1, beta_2=0.001, epsilon=1e-08, decay=decay_rate)

In [6]:
# LEFT BLOCK (to analyse amino acid sequences)
input_seq = Input(shape=(max_len_prot, len(prot_dict)), dtype='float32')
conv_seq = Conv1D(filters=64, padding='same', strides=1, kernel_size=3, activation='relu')(input_seq)
dropout_1 = Dropout(0.4)(conv_seq)
flatten_seq = Flatten()(dropout_1)#(dense_seq)
dense_seq_2 = Dense(50)(flatten_seq)
dropout_2 = Dropout(0.4)(dense_seq_2)

#RIGHT BRANCH (to analyse fingerprints)
input_fps = Input(shape=(881,), dtype='float32')
dense_fps = Dense(50)(input_fps)
dropout_3 = Dropout(0.4)(dense_fps)
#bn_3 =  BatchNormalization()(dense_fps)#(dense_seq_2)#(conv_seq)


#MERGE BOTH BRANCHES
main_merged = concatenate([dropout_2, dropout_3],axis=1)#([dense_seq_2, dense_fps], axis=1)

main_dense = Dense(2, activation='softmax')(main_merged)

#build and compile model
model = Model(inputs=[input_seq, input_fps], outputs=[main_dense])
model.compile(loss='categorical_crossentropy', optimizer = adamm, metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1499, 26)     0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 1499, 64)     5056        input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1499, 64)     0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 95936)        0           dropout_1[0][0]                  
____________________________________________________________________________________________

In [ ]:
for fold in range(nfolds): 
    print("Fold:", str(fold))
    #training
    file_train = "".join((absPath, "data/", protein_type, "/resampling_after_clustering/",
                                 str(fold), "/compounds_activity_training.h5"))
    f_train = h5py.File(file_train, 'r')
    group = '/activity'
    table = "prot_comp"
    #validation
    file_val = "".join((absPath, "data/", protein_type, "/resampling_after_clustering/",
                                 str(fold), "/compounds_activity_validation.h5"))
    f_val = h5py.File(file_val, 'r')
    #test
    file_test = "".join((absPath, "data/", protein_type, "/resampling_after_clustering/",
                                 str(fold), "/compounds_activity_test.h5"))
    f_test = h5py.File(file_test, 'r')
    
    sample_indices_train = range(len(f_train[group][table]))
    sample_indices_val = range(len(f_val[group][table]))
    sample_indices_test = range(len(f_test[group][table]))
    
    #Defining generators
    train_generator = batch_generator_DL(batch_size, f_train, group, table, sample_indices_train, 
                                     max_len_prot, type_padding_prot=type_padding_prot)
    val_generator = batch_generator_DL(batch_size, f_val, group, table, sample_indices_val, 
                                     max_len_prot, type_padding_prot=type_padding_prot)
    
    #defining callbacks
    if not os.path.exists("".join((absPath, "data/", protein_type, 
                                   "/resampling_after_clustering/logs/", str(fold), "/"))):
        os.makedirs("".join((absPath, "data/", protein_type, 
                                   "/resampling_after_clustering/logs/", str(fold), "/")))
    
    log_path = "".join((absPath, "data/", protein_type, 
                                   "/resampling_after_clustering/logs/", str(fold), "/training_log.csv"))
    csv_logger = CSVLogger(log_path)

    if not os.path.exists("".join((absPath, "data/", protein_type, 
                                   "/resampling_after_clustering/checkpoint/", str(fold), "/"))):
        os.makedirs("".join((absPath, "data/", protein_type, 
                                   "/resampling_after_clustering/checkpoint/", str(fold), "/")))

    #if there are already files in the folder, it removes them
    r = glob("".join((absPath, "data/", protein_type, 
                                   "/resampling_after_clustering/checkpoint/", str(fold), "/*")))
    for i in r:
        os.remove(i)
   
    terminan = TerminateOnNaN()
    checkpoint_path = "".join((absPath, "data/", protein_type, 
                                   "/resampling_after_clustering/checkpoint/", str(fold),
                               "/weights-improvement-{epoch:03d}-{val_accuracy:.4f}.hdf5"))
    mcheckpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=0, 
                                          save_best_only=True, save_weights_only=False)

    callbacks_list = [csv_logger, terminan, mcheckpoint ]
    print("Training")
    # fitting the model
    history = model.fit_generator(generator=train_generator, 
                              validation_data=val_generator,
                             steps_per_epoch= int(len(sample_indices_train)/batch_size),
                              validation_steps=int(len(sample_indices_val)/batch_size),
                             epochs=epochss,
                             callbacks=callbacks_list,
                             verbose=1)
    #saving history
    if not os.path.exists("".join((absPath, "data/", protein_type, "/resampling_after_clustering/results/", 
                                   str(fold), "/"))):
        os.makedirs("".join((absPath, "data/", protein_type, "/resampling_after_clustering/results/", str(fold), "/")))

    with open("".join((absPath, "data/", protein_type, "/resampling_after_clustering/results/", str(fold), "/history.pickle")), 'wb') as handle:
        pickle.dump(history, handle)
        
    print("Prediction on test data")
    #PROTEINS
    batch_sequences = list(f_test[group][table][sample_indices_test]["sequence"])
    #COMPOUNDS
    batch_compounds = list(f_test[group][table][sample_indices_test]["fingerprint"])
    #LABELS
    batch_y = list(f_test[group][table][sample_indices_test]["label"])
    #processing sequences and compounds
    seqs_onehot = np.asarray(processing_sequences(batch_sequences, max_len_prot, type_padding_prot))
    comps_batch = np.asarray(processing_fingerprints(batch_compounds))
    batch_labels = np.asarray(bin_to_onehot(batch_y))
    
    history_path = "".join((absPath, "data/", protein_type, "/resampling_after_clustering/results/", 
                                   str(fold), "/history.pickle"))
    path_to_confusion = "".join((absPath, "data/", protein_type, "/resampling_after_clustering/results/", 
                                   str(fold), "/"))
    path_to_auc = "".join((absPath, "data/", protein_type, "/resampling_after_clustering/results/", 
                                   str(fold), "/"))
    
    history = plot_history(history_path, "".join((absPath, "data/", protein_type, 
                                                  "/resampling_after_clustering/results/", 
                                   str(fold), "/")))
    path_to_cp = ''.join((absPath, "data/", protein_type, "/resampling_after_clustering/checkpoint/", 
                          str(fold), "/"))

    model, best_path = load_best_model(history, path_to_cp)

    cps_loc = ''.join((absPath, "data/", protein_type, "/resampling_after_clustering/checkpoint/", 
                          str(fold), "/*.hdf5")) 

    #removing the rest of weights
    fileList = glob(cps_loc, recursive=True)
    fileList.remove(best_path)
    if len(fileList) >1:
        for filePath in fileList:
            try:
                os.remove(filePath)
            except OSError:
                print("Error while deleting file")
    
    y_predprob = model.predict([seqs_onehot, comps_batch])
    y_prob = y_predprob[:,1]
    y_pred = y_predprob.argmax(-1)
    y_test = batch_labels.argmax(-1)
    print("Counting predicted: ", Counter(y_pred))
    
    batch_compID_test = list(f_test[group][table][sample_indices_test]["da_comp_id"])
    batch_protID_test = list(f_test[group][table][sample_indices_test]["da_prot_id"])
    
    #confusion matrix
    confusion_matrix(y_test, y_pred, path_to_confusion)
        
    #AUC
    file_auc = ''.join((absPath, "data/", protein_type, "/resampling_after_clustering/results/", 
                                   str(fold), "/AUC.pickle"))
    compute_roc(y_test, y_prob, path_to_auc)
    
    # saving predictions on test set

    predictions_test = pd.DataFrame({"y_test":y_test, "y_prob":y_prob, "y_pred":y_pred, "comp_ID": batch_compID_test,
                                "DeepAffinity Protein ID": batch_protID_test})

    if not os.path.exists("".join((absPath, "data/", protein_type, "/resampling_after_clustering/predictions/", str(fold), "/"))):
        os.makedirs("".join((absPath, "data/", protein_type, "/resampling_after_clustering/predictions/", str(fold), "/")))

    predictions_test.to_csv("".join((absPath, "data/", protein_type, "/resampling_after_clustering/predictions/", str(fold), "/test.csv")))
    f_train.close()
    f_val.close()
    f_test.close()

Fold: 0
Training
Epoch 1/100
1873/1873 [==============================] - 155s 83ms/step - loss: 272.7694 - accuracy: 0.5620 - val_loss: 179.1213 - val_accuracy: 0.5807
Epoch 2/100
1873/1873 [==============================] - 148s 79ms/step - loss: 210.0521 - accuracy: 0.5616 - val_loss: 158.3353 - val_accuracy: 0.5545: 38s - loss: 247.6323 - ac - ETA: 37s - loss
Epoch 3/100
1873/1873 [==============================] - 150s 80ms/step - loss: 196.5569 - accuracy: 0.5874 - val_loss: 63.2100 - val_accuracy: 0.5695
Epoch 4/100
1873/1873 [==============================] - 150s 80ms/step - loss: 220.5324 - accuracy: 0.5962 - val_loss: 40.5251 - val_accuracy: 0.6041
Epoch 5/100
1873/1873 [==============================] - 150s 80ms/step - loss: 713.9580 - accuracy: 0.6035 - val_loss: 246.6441 - val_accuracy: 0.5194 - ETA:  - ETA: 21s - loss: 772.4265 - ETA: 1s - los
Epoch 6/100
1873/1873 [==============================] - 150s 80ms/step - loss: 1450.8143 - accuracy: 0.5082 - val_loss: 1741.23

Fold: 1
Training
Epoch 1/100
1897/1897 [==============================] - 147s 78ms/step - loss: 0.7031 - accuracy: 0.6378 - val_loss: 0.1532 - val_accuracy: 0.6697
Epoch 2/100
1897/1897 [==============================] - 144s 76ms/step - loss: 0.7052 - accuracy: 0.6390 - val_loss: 0.8619 - val_accuracy: 0.6705
Epoch 3/100
1897/1897 [==============================] - 144s 76ms/step - loss: 0.7057 - accuracy: 0.6413 - val_loss: 0.4957 - val_accuracy: 0.67860.7079 - ac - ETA: 4s - loss: 0.7081 - accuracy:  - ETA: 4s - loss: 0.7071 - accuracy - ETA: 4s - loss: 0.7074 - accuracy: 0.64 - ETA: 4s - - ETA: 3s - loss: 0.7078 - accuracy: 0. - ETA: 2s - loss: 0.7079 -  - ETA: 2s - loss: 0.707
Epoch 4/100
1897/1897 [==============================] - 146s 77ms/step - loss: 0.7044 - accuracy: 0.6425 - val_loss: 0.6367 - val_accuracy: 0.6679loss: 0.7069 - ac - ETA: 3s - loss: 0.7070 - accuracy - ETA: 3s - loss: 0.7074 - ac
Epoch 5/100
1897/1897 [==============================] - 146s 77ms/step - los

1897/1897 [==============================] - 145s 77ms/step - loss: 0.6485 - accuracy: 0.6606 - val_loss: 0.8060 - val_accuracy: 0.6395
Epoch 77/100
1897/1897 [==============================] - 147s 77ms/step - loss: 0.6485 - accuracy: 0.6593 - val_loss: 0.9989 - val_accuracy: 0.6474
Epoch 78/100
1897/1897 [==============================] - 146s 77ms/step - loss: 0.6495 - accuracy: 0.6587 - val_loss: 1.1808 - val_accuracy: 0.6653
Epoch 79/100
1897/1897 [==============================] - 146s 77ms/step - loss: 0.6512 - accuracy: 0.6572 - val_loss: 0.9230 - val_accuracy: 0.6627
Epoch 80/100
1897/1897 [==============================] - 146s 77ms/step - loss: 0.6487 - accuracy: 0.6599 - val_loss: 0.4999 - val_accuracy: 0.6667
Epoch 81/100
1897/1897 [==============================] - 146s 77ms/step - loss: 0.6504 - accuracy: 0.6580 - val_loss: 0.6731 - val_accuracy: 0.6629
Epoch 82/100
1897/1897 [==============================] - 145s 76ms/step - loss: 0.6471 - accuracy: 0.6611 - val_loss: 

1914/1914 [==============================] - 152s 79ms/step - loss: 0.6513 - accuracy: 0.6598 - val_loss: 0.6559 - val_accuracy: 0.6478
Epoch 74/100
1914/1914 [==============================] - 152s 79ms/step - loss: 0.6515 - accuracy: 0.6593 - val_loss: 0.4236 - val_accuracy: 0.6503- accuracy:  - ETA: 41s - loss: - ETA: 3 - ETA: 29
Epoch 75/100
1914/1914 [==============================] - 151s 79ms/step - loss: 0.6510 - accuracy: 0.6590 - val_loss: 0.4208 - val_accuracy: 0.6511
Epoch 76/100
1914/1914 [==============================] - 151s 79ms/step - loss: 0.6504 - accuracy: 0.6594 - val_loss: 0.5531 - val_accuracy: 0.6581
Epoch 77/100
1914/1914 [==============================] - 151s 79ms/step - loss: 0.6490 - accuracy: 0.6591 - val_loss: 0.6213 - val_accuracy: 0.6471
Epoch 78/100
1914/1914 [==============================] - 152s 79ms/step - loss: 0.6499 - accuracy: 0.6595 - val_loss: 0.7641 - val_accuracy: 0.6506
Epoch 79/100
1914/1914 [==============================] - 152s 79ms/s

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1893/1893 [==============================] - 147s 78ms/step - loss: 0.6412 - accuracy: 0.6577 - val_loss: 0.4200 - val_accuracy: 0.6383
Epoch 83/100
1893/1893 [==============================] - 148s 78ms/step - loss: 0.6408 - accuracy: 0.6577 - val_loss: 0.8148 - val_accuracy: 0.6335 ETA: 28s - loss: 0.6489 -  - ETA: 21s - loss:  - ETA: 18s - - ETA:  - ETA: 0s - loss: 0.6409 - accura
Epoch 84/100
1893/1893 [==============================] - 148s 78ms/step - loss: 0.6396 - accuracy: 0.6579 - val_loss: 0.6234 - val_accuracy: 0.6353oss: 0.6435  - ETA: 48s - loss: 0.6432 - accuracy: 0 - ETA: 41s - loss: 0.6449 - accuracy: - ETA:  - 
Epoch 85/100
1893/1893 [==============================] - 148s 78ms/step - loss: 0.6393 - accuracy: 0.6589 - val_loss: 1.0947 - val_accuracy: 0.6346
Epoch 86/100
1893/1893 [==============================] - 150s 79ms/step - loss: 0.6419 - accuracy: 0.6586 - val_loss: 0.3475 - val_accuracy: 0.6387ETA: 1s - loss: 0 - ETA: 0s - loss: 0.6421 - accuracy: 0.65 - ETA:

1892/1892 [==============================] - 151s 80ms/step - loss: 0.6352 - accuracy: 0.6572 - val_loss: 0.8840 - val_accuracy: 0.6605
Epoch 81/100
1892/1892 [==============================] - 151s 80ms/step - loss: 0.6338 - accuracy: 0.6584 - val_loss: 0.8268 - val_accuracy: 0.6593s - loss: 0.6337 - accuracy: 
Epoch 82/100
1892/1892 [==============================] - 151s 80ms/step - loss: 0.6336 - accuracy: 0.6581 - val_loss: 1.1529 - val_accuracy: 0.6663
Epoch 83/100
1892/1892 [==============================] - 151s 80ms/step - loss: 0.6332 - accuracy: 0.6588 - val_loss: 0.4338 - val_accuracy: 0.6675
Epoch 84/100
1892/1892 [==============================] - 151s 80ms/step - loss: 0.6335 - accuracy: 0.6585 - val_loss: 1.2980 - val_accuracy: 0.6622
Epoch 85/100
1892/1892 [==============================] - 151s 80ms/step - loss: 0.6328 - accuracy: 0.6586 - val_loss: 0.3903 - val_accuracy: 0.6670s - los
Epoch 86/100
1892/1892 [==============================] - 152s 80ms/step - loss: 0.

1890/1890 [==============================] - 152s 80ms/step - loss: 0.6332 - accuracy: 0.6595 - val_loss: 0.4661 - val_accuracy: 0.6552
Epoch 81/100
1890/1890 [==============================] - 152s 80ms/step - loss: 0.6317 - accuracy: 0.6617 - val_loss: 0.2210 - val_accuracy: 0.6498
Epoch 82/100
1890/1890 [==============================] - 153s 81ms/step - loss: 0.6315 - accuracy: 0.6613 - val_loss: 0.2277 - val_accuracy: 0.6527
Epoch 83/100
1890/1890 [==============================] - 153s 81ms/step - loss: 0.6307 - accuracy: 0.6614 - val_loss: 0.2332 - val_accuracy: 0.6538
Epoch 84/100
1890/1890 [==============================] - 154s 81ms/step - loss: 0.6327 - accuracy: 0.6597 - val_loss: 1.0143 - val_accuracy: 0.6529
Epoch 85/100
1890/1890 [==============================] - 153s 81ms/step - loss: 0.6329 - accuracy: 0.6605 - val_loss: 0.6908 - val_accuracy: 0.6547
Epoch 86/100
1890/1890 [==============================] - 152s 80ms/step - loss: 0.6320 - accuracy: 0.6620 - val_loss: 

1907/1907 [==============================] - 147s 77ms/step - loss: 0.6245 - accuracy: 0.6647 - val_loss: 0.3793 - val_accuracy: 0.6431
Epoch 73/100
1907/1907 [==============================] - 147s 77ms/step - loss: 0.6238 - accuracy: 0.6647 - val_loss: 0.8248 - val_accuracy: 0.6463
Epoch 74/100
1907/1907 [==============================] - 147s 77ms/step - loss: 0.6239 - accuracy: 0.6647 - val_loss: 0.7587 - val_accuracy: 0.6472
Epoch 75/100
1907/1907 [==============================] - 147s 77ms/step - loss: 0.6251 - accuracy: 0.6631 - val_loss: 0.7734 - val_accuracy: 0.6467
Epoch 76/100
1907/1907 [==============================] - 147s 77ms/step - loss: 0.6251 - accuracy: 0.6645 - val_loss: 1.0650 - val_accuracy: 0.6500
Epoch 77/100
1907/1907 [==============================] - 147s 77ms/step - loss: 0.6229 - accuracy: 0.6656 - val_loss: 0.9486 - val_accuracy: 0.6436
Epoch 78/100
1907/1907 [==============================] - 147s 77ms/step - loss: 0.6259 - accuracy: 0.6645 - val_loss: 